In [58]:
import numpy as np
import cv2
import math
cap = cv2.VideoCapture("Many_people (2).mp4")

In [59]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
if frame is not None:
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    if frame is None:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [42]:
ab=good_new-good_old

In [54]:
print("Total detected persons:",len(ab))
n=0
s=0
e=0
w=0
for i in range(0,len(ab)):
        tan_theta=ab[i][1]/ab[i][0]
        theta=math.degrees(math.atan(tan_theta))
        if(theta<0):
            theta=360+theta
        if 135 > theta > 45:
             n=n+1
                
        elif 315 > theta > 225:
             s=s+1
        
        elif 225 > theta > 135:
             e=e+1
        
        elif 360 > theta > 315 or 45 > theta > 0:
             w=w+1
print("Total People heading towards North:",n)
print("Total People heading towards South:",s)
print("Total People heading towards East:",e)
print("Total People heading towards West:",w)

Total detected persons: 85
Total People heading towards North: 11
Total People heading towards South: 26
Total People heading towards East: 0
Total People heading towards West: 48
